## Classification-tree

- Sequence of if-ealse questions about individual features
- <u>Objective</u>: infer class labels
- Able to capture non-linear relationships between features and labels
- Don't require feature scaling

## Decision-tree diagram

<img src='decision-tree-diagram.PNG'>

~~~
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1)

dt = DecisionTreeClassifier(max_depth=2, random_state=1)

dt.fit(X_train, y_train)

y_pred = dt.predict(X_test)

accuracy_score(y_test, y_pred)
~~~

## Decision regions

- <u>Decision region</u>: region in the feature space where all instaces are assigned to one class label.
- <u>Decision boundary</u>: surface separating different decision regions.

## Building blocks

- <u>Decision-Tree</u>: data structure consisting of a hierarchy of nodes
- <u>Node</u>: question or prediction
	- Root: *no* parent node, question giving rise to *two* children nodes
	- Internal node: *one* parent node, question giving rise to *two* children nodes
	- Leaf: *one* parent node, *no* children nodes -> prediction

## Information gain

$$
IG (f, sp) = I (\textrm{parent}) - \bigg( \displaystyle\frac{N_{\textrm{left}}}{N} I(\textrm{left}) + \displaystyle\frac{N_{\textrm{right}}}{N} I(\textrm{right}) \bigg)
$$

where $f$ means feature and $sp$, split-point.

- Criteria to measure impurity of a node $I(\textrm{node})$:
	- gini index
	- entropy

## Classification-tree learning

- Nodes are grown recursively
- At each node, split the data based on:
	- feature $f$ and split-point $sp$ to maximize $IG(\textrm{node})$
	- if $IG (\textrm{node}) = 0$, declare the node a leaf

~~~
dt = DecisionTreeClassifier(criterion='gini', random_state=1)
~~~

## Regression-tree

~~~
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

dt = DecisionTreeRegressor(max_depth=4, min_samples_leaf=0.1, random_state=3)

dt.fit(X_train, y_train)

y_pred = dt.predict(X_test)

mse_dt = MSE(y_test, y_pred)

rmse_dt = mse_dt**(1/2)

print(rmse_dt)
~~~

## Information criterion for Regression-tree

$$
I(\textrm{node}) = \underbrace{MSE(\textrm{node})}_{\textrm{mean-squared-error}} = \displaystyle\frac{1}{N_{\textrm{node}}} \displaystyle\sum_{i\ \in\ \textrm{node}} \big( y^{(i)} - \hat y_{\textrm{ node}} \big)^2 \\
\underbrace{\hat y_{\textrm{node}}}_{\textrm{mean-target-value}} = \displaystyle\frac{1}{N_{\textrm{node}}} \displaystyle\sum_{i\ \in\ \textrm{node}} y^{(i)}
$$

<img src='./IMAGES/lin-reg-tree-reg.PNG'>

## Supervised learning: Under the hood

- Supervised learning: $y = f(x)$, $f$ is unknown
	- Find a model $\hat f$ that best approximates $f$ : $\hat f \apporx f$
	- $\hat f$ can be Logistic Regression, Decision Tree, Neural Network...
	- Discard noise as much as possible
- <u>End goal</u>: $\hat f$ should achieve a low predictive error on unseen data

## Difficulties in approximating $f$

- <u>Overfitting</u>: $\hat f (x)$ fits the training set noise
- <u>Undertraining</u>: $\hat f$ is not flexible enough to approximate $f$

## Generalization error

- Does $\hat f$ generalize well on unseen data?
- It can be decomposed as follows: Generalization Error of $\hat f = \textrm{ bias}^2 + \textrm{ variance } + \textrm{ irreducible error}$

- Bias: error term that tells you, on average, how much $\hat f \neq f$
- Variance: tells you how much $\hat f$ is inconsistent over different training sets

- <u>Model complexity</u>: sets the flexibility of $\hat f$

<img src='./IMAGES/bias-variance-tradeoff.PNG'>

## Estimating the Generalization Error

- How do we estimate the generalization error of a model?
	- Cannot be done directly because:
		- $f$ is unknown
		- usually you only have one dataset
		- noise is unpredictable
- Solution:
	- split the data to training and test sets
	- fit $\hat f$ to the training set
	- evaluate the error of $\hat f$ on the *unseen* test set
	- generalization error of $\hat f \approx$ test error of $\hat f$

## Better Model Evaluation with Cross-Validation (CV)

- Test set should not be touched until we are confident the performance of $\hat f$
- Evaluating $\hat f$ on training set: biased estimate, $\hat f$ has already seen all training points
	- Solution: Cross-validation (CV)
		- K-Fold CV
		- Hold-out CV

<img src='./IMAGES/kfold-cv.PNG'>

$$
\textrm{CV error } = \displaystyle\frac{E_1 + ... + E_10}{10}
$$

## Diagnose variance problems

- If $\hat f$ suffers from <u>high variance</u>: CV error of $\hat f >$ training set error of $\hat f$ .
	- $\hat f$ is said to overfit the training set.
		- decrease model complexity
		- for instance: decrease `max_depth`, increase `min_samples_leaf`
		- gather more data

## Diagnose bias problems

- If $\hat f$ suffers from <u>high bias</u>: CV error of $\hat f \approx$ training set error of $\hat f \gg$ desired error
	- $\hat f$ is said to underfit the training set
		- increase model complexity
		- for instance: increase `max_depth`, decrease `min_samples_leaf`
		- gather more relevant features

## K-Fold CV in scikit-learn

~~~
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import cross_val_score

SEED = 123

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=SEED)

dt = DecisionTreeRegressor(max_depth=4, min_samples_leaf=0.14, random_state=SEED)

MSE_CV = cross_val_score(dt, X_train, y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1)

dt.fit(X_train, y_train)

y_pred_train = dt.predict(X_train)
y_pred_test = dt.predict(X_test)
~~~

## Limitations of CARTs

- Classification: can only produce orthogonal decision boundaries
- Sensitive to small variations in the training set
- High variance: unconstrained CARTs may overfit the training set
- Solution: ensemble learning

## Ensemble Learning

- Train different models on the same dataset
- Let each model make its predictions
- Meta-model: aggregates predictions of individual models
- Final prediction: more robust and less prone to errors
- Best results: models are skillful in different ways

<img src='./IMAGES/ensemble-learning.PNG'>

## Voting classifier

- Binary classification task
- $N$ classifiers make predictions: $P_1, P_2, ..., P_N$ with $P_i \in \{0,1\}$

~~~
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.ensemble import VotingClassifier

SEED = 123

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=SEED)

lr = LogisticRegression(random_state=SEED)
knn = KNN()
dt = DecisionTreeClassifier(random_state=SEED)

classifiers = [('Logistic Regression', lr),
		('K Nearest Neighbours', knn),
		('Classification Tree', dt)]

for clf_name, clf in classifiers:
	clf.fit(X_train, y_train)

	y_pred = clf.predict(X_test)

	print('{:s} : {:.3f}'.format(clf_name, accuracy_score(y_test, y_pred)))

vc = VotingClassifier(estimators=classifiers)

vc.fit(X_train, y_train)
y_pred = vc.predict(X_test)

print('Voting classifier: {.3f}'.format(accuracy_score(y_test, y_pred)))
~~~


## Ensemble methods

- Voting classifier: same training set, different algorithms
- Bagging: one algorithm, different subsets of the training set

## Bagging

- stands for *Bootstrap Aggregation*
- uses a technique known as bootstrap
- reduces variance of individual models in the ensemble

## Bagging: classification & regression

- Classification:
	- Aggregates predictions by majority voting
	- `BaggingClassifier` in scikit-learn
- Regression:
	- Aggregates predictions through averaging
	- `BaggingRegressor` in scikit-learn

~~~
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

SEED = 1

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=SEED)

dt = DecisionTreeClassifier(max_depth=4, min_samples_leaf=0.16, random_state=SEED)

bc = BaggingClassifier(base_estimator=dt, n_estimator=300, n_jobs=-1)

bc.fit(X_train, y_train)

y_pred = bc.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
~~~

## Out of Bag (OOB) instances

- On average, for eahc model, 63% of the training instances are sampled
- The remaining 37% constitute the OOB instances

$$
OOB_{\textrm{score}} = \displaystyle\frac{OOB_1 + ... + OOB_N}{N}
$$

~~~
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

SEED = 1

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=SEED)

dt = DecisionTreeClassifier(max_depth=4, min_sample_leaf=0.16, random_state=SEED)

bc = BaggingClassifier(base_estimator=dt, n_estimator=300, oob_score=True, n_jobs=-1)

bc.fit(X_train, y_train)

y_pred = bc.predict(X_test)

test_accuracy = accuracy_score(y_test, y_pred)

oob_accuracy = bc.oob_score_
~~~

## Random Forests

- Base estimator: decision tree
- each estimator: trained on a different bootstrap sample having the same size as the training set
- introduces further randomization in the training of individual trees
- $d$ features are samples at each node without replacement ($d < $ total number of features)

## RF: classification & regression

- Classification:
	- Aggregates predictions by majority voting
	- `RandomForestClassifier` in scikit-learn
- Regression
	- Aggregates predictions thorugh averaging
	- `RandomForestRegressor` in scikit-learn

~~~
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE

SEED = 1

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=SEED)

rf = RandomForestClassifier(n_estimators=400, min_samples_leaf=0.12, random_state=SEED)

rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

rmse_test = MSE(y_test, y_pred)**(1/2)
~~~

## Feature importance

Tree-based methods: enable measuring the importance of each feature in prediction.

In `sklearn`:
- how much the tree nodes use a particular feature (weighted average) to reduce impurity
- accessed using the attribute `feature_importances_`

~~~
importance_rf = pd.Series(rf.feature_importances_, index=X.columns)

sorted_importances_rf = importances_rf.sort_values()

sorted_importances_rf.plot(kind='barh', color='lightgreen')
plt.show()
~~~


## Boosting

- Boosting: ensemble method combining several weak learners to form a strong learner
- Weak learner: model doing slightly better than random guessing
	- Example: decision stump (CART whose `max_depth` is $1$)

- Train an ensemble of predictors sequentially
- Each predictor tries to correct its predecessor

## AdaBoost

- Stands for Adaptive Boosting
- Each predictor pays morea attention to the instances wrongly predicted by its predecessor
- Achieved by changing the weights of training instances

<img src='./IMAGES/adaboost-training.PNG'>

## AdaBoost: prediction

- Classification
	- Weighted majority voting
	- In scikit-learn: `AdaBoostClassifier`
- Regression:
	- Weighted average
	- In scikit-learn: `AdaBoostRegressor`


In [1]:
import pandas as pd

In [2]:
wbc = pd.read_csv('./DATASETS/wbc.csv', usecols=range(32))

In [3]:
wbc.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [4]:
X = wbc.loc[:,'radius_mean':'fractal_dimension_worst']

In [5]:
y = wbc.diagnosis

In [6]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

SEED = 1

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=SEED)

In [7]:
dt = DecisionTreeClassifier(max_depth=1, random_state=SEED)

adb_clf = AdaBoostClassifier(base_estimator=dt, n_estimators=100)

adb_clf.fit(X_train, y_train)

# Predict the test set probabilities of positive class
y_pred_proba = adb_clf.predict_proba(X_test)[:,1]

adb_clf_roc_auc_score = roc_auc_score(y_test, y_pred_proba)

In [8]:
print('ROC AUC score: {:.2f}'.format(adb_clf_roc_auc_score))

ROC AUC score: 0.99


## Gradient Boosting (GB)

- Sequential correction of predecessor's errors
- Does not tweak the weights of training instances
- Each predictor is trained using the residual errors of its predecessor as labels
- Gradient Boosted Trees: a CART is used as a base learner

<img src='./IMAGES/gb-trees-training.PNG'>

## GB Trees: prediction

- Regression:
	- $y_{\textrm{pred}} = y_1 + \eta r_1 + ... + \eta r_N$
	- In scikit-learn: `GradientBoostingRegressor`
- Classification:
	- In scikit-learn: `GradientBoostingClassifier`


In [9]:
auto = pd.read_csv('./DATASETS/auto.csv')

In [10]:
auto.head()

,mpg,displ,hp,weight,accel,origin,size
0,18.0,250.0,88,3139,14.5,US,15.0
1,9.0,304.0,193,4732,18.5,US,20.0
2,36.1,91.0,60,1800,16.4,Asia,10.0
3,18.5,250.0,98,3525,19.0,US,15.0
4,34.3,97.0,78,2188,15.8,Europe,10.0


In [11]:
X = auto.drop(['mpg','origin'], axis=1)
y = auto.mpg

In [12]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error as MSE

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=SEED)

In [14]:
gbt = GradientBoostingRegressor(n_estimators=300, max_depth=1, random_state=SEED)

gbt.fit(X_train, y_train)

y_pred = gbt.predict(X_test)

rmse_test = MSE(y_test, y_pred)**(1/2)

print('Test set RMSE: {:.2f}'.format(rmse_test))

Test set RMSE: 4.08


## Gradient Boosting: cons

- GB involves an exhaustive search procedure
- Each CART is trained to find the best split points and features
- May lead to CARTs using the same split points and maybe the same features

## Stochastic Gradient Boosting (SGB)

- each tree trained on a random subset of rows of the training dataset
- the sampled instances (40%-80% of the training set) are sampled without replacement
- features sampled (without replacement) when choosing split points
- Result: further ensemble diversity
- Effect: adding further variance to the ensemble of trees

<img src='./IMAGES/sgb-training.PNG'>


In [15]:
sgbt = GradientBoostingRegressor(max_depth=1, subsample=0.8, max_features=0.2, n_estimators=300, random_state=SEED)

sgbt.fit(X_train, y_train)

y_pred = sgbt.predict(X_test)

rmse_test = MSE(y_test, y_pred)**(1/2)

print('Test set RMSE: {:.2f}'.format(rmse_test))

Test set RMSE: 4.28


## What is hyperparameter tuning?

- <u>Problem</u>: search for a set of optimal hyperparameters for a learning algorithm
- <u>Solution</u>: find a set of optimal hyperparameters that results in an optimal model
- <u>Optimal model</u>: yields an optimal score
- <u>Score</u>: in scikit-learn defaults to accuracy (classification) and $R^2$ (regression)
- Cross validation is used to estimate the generalization performance

## Approaches to hyperparameter tuning

- Grid search
- Random search
- Bayesian optimization
- Genetic algorithms
- ...

## Grid search cross validation

- Manually set a grid of discrete hyperparameter values
- Set a metric for scoring model performance
- Search exhaustively through the grid
- For each set of hyperparameters, evaluate each model's CV score
- The optimal hyperparameters are those of the model achieving the best CV score

## Grid search: example
- Hyperparameter grids:
	- `max_depth` = $\{2,3,4\}$
	- `min_samples_leaf` = $\{0.05, 0.1\}$
- Hyperparameter space = $\{(2, 0.05), (2, 0.1), (3, 0.05), ...\}$
- CV scores = $\{\textrm{score}_{(2, 0.05)}, ...\}$
- Optimal hyperparameters: set of hyperparameters corresponding to the best CV score


In [16]:
dt = DecisionTreeClassifier(random_state=SEED)

print(dt.get_params())

{'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'presort': False, 'random_state': 1, 'splitter': 'best'}


In [17]:
X = wbc.loc[:,'radius_mean':'fractal_dimension_worst']
y = wbc.diagnosis

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=SEED)

In [19]:
from sklearn.model_selection import GridSearchCV

params_dt = {'max_depth': [3, 4, 5, 6], 'min_samples_leaf': [0.04, 0.06, 0.08], 'max_features': [0.2, 0.4, 0.6, 0.8]}

grid_dt = GridSearchCV(estimator=dt, param_grid=params_dt, scoring='accuracy', cv=10, n_jobs=-1)

grid_dt.fit(X_train, y_train)

best_hyperparams = grid_dt.best_params_

print('Best hyperparameters: \n', best_hyperparams)

Best hyperparameters: 
 {'max_depth': 4, 'max_features': 0.4, 'min_samples_leaf': 0.04}


C:\Users\BigData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [20]:
best_model = grid_dt.best_estimator_

test_acc = best_model.score(X_test, y_test)

print('Test set accuracy of best model: {:.3f}'.format(test_acc))

Test set accuracy of best model: 0.947


---

In [21]:
X = auto.drop(['mpg','origin'], axis=1)
y = auto.mpg

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

In [23]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=SEED)

print(rf.get_params())

{'bootstrap': True, 'criterion': 'mse', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 'warn', 'n_jobs': None, 'oob_score': False, 'random_state': 1, 'verbose': 0, 'warm_start': False}


In [24]:
params_rf = {'n_estimators': [300, 400, 500], 'max_depth': [4, 6, 8], 
             'min_samples_leaf': [0.1, 0.2], 'max_features': ['log2', 'sqrt']}

grid_rf = GridSearchCV(estimator=rf, param_grid=params_rf, cv=3, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)

grid_rf.fit(X_train, y_train)

Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed:   10.8s finished
C:\Users\BigData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=1, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': [300, 400, 500], 'max_depth': [4, 6, 8], 'min_samples_leaf': [0.1, 0.2], 'max_features': ['log2', 'sqrt']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=1)

In [25]:
best_hyperparams = grid_rf.best_params_

print('Best hyperparameters: \n', best_hyperparams)

Best hyperparameters: 
 {'max_depth': 4, 'max_features': 'log2', 'min_samples_leaf': 0.1, 'n_estimators': 300}


In [26]:
best_model = grid_rf.best_estimator_

y_pred = best_model.predict(X_test)

rmse_test = MSE(y_test, y_pred)**(1/2)

print('Test set RMSE of rf: {:.2f}'.format(rmse_test))

Test set RMSE of rf: 3.92
